## Generación de base de datos de palabras en Español

In [1]:
# librerias utilizadas
import pandas as pd
import nltk
import string
import datetime
from collections import Counter
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /home/jogabell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jogabell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [147]:
# Lectura de base de datos
gh_2015=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2015',encode='latin1')
gh_2016=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2016',encode='latin1')
gh_2017=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2017',encode='latin1')
gh_2018=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2018',encode='latin1')
gh_2019=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2019',encode='latin1') 

In [148]:
#gh_2015.head

In [149]:
features=[u'Project ID', u'Name', u'Organization',u'Published Status',
       u'Goal', u'Percentage Raised', u'Number of Individual Donors',  
       u'Short DescSP', u'Long Desc SP']
_2015=gh_2015[features]
_2016=gh_2016[features]
_2017=gh_2017[features]
_2018=gh_2018[features]
_2019=gh_2019[features]

In [150]:
_2015['Published Status'].value_counts()

ended       175
draft        87
private       3
rejected      1
Name: Published Status, dtype: int64

In [151]:
# Limpiado base de datos. Se quitando registros con 'Published Status'='draft' o 'rejected'

data=_2015.append(_2016).append(_2017).append(_2018).append(_2019)

In [152]:
data.shape

(3966, 9)

In [153]:
mask1=(data['Published Status']!= 'draft')

In [154]:
data['Published Status'].value_counts()

draft        3105
ended         749
rejected       70
publish        28
private        10
submitted       2
archived        1
Name: Published Status, dtype: int64

In [155]:
data=data[mask1]

In [156]:
data['Published Status'].value_counts()

ended        749
rejected      70
publish       28
private       10
submitted      2
archived       1
Name: Published Status, dtype: int64

In [157]:
mask2=(data['Published Status']!= 'rejected')

In [158]:
data=data[mask2]

In [159]:
data['Published Status'].value_counts()

ended        749
publish       28
private       10
submitted      2
archived       1
Name: Published Status, dtype: int64

## Creación de base de datos de texto para proyectos que no recaudaron

In [160]:
mask3=data[u'Percentage Raised']<=50

In [161]:
data_cero_donation=data[mask3]

In [162]:
mask4=data[u'Number of Individual Donors']<=0

In [163]:
data_cero_donation=data[mask4]

In [164]:
data_cero_donation.shape

(150, 9)

## Creación de base de datos de texto para proyectos que alcanzaron o rebasaron la meta

In [165]:
mask5=data['Percentage Raised']>=100

In [166]:
data_goal_donation=data[mask5]

In [167]:
mask6=data_goal_donation['Number of Individual Donors']>=1

In [168]:
data_goal_donation=data_goal_donation[mask6]

In [169]:
data_goal_donation.shape

(134, 9)

## Función para preparar los datos de texto

In [186]:
def text_analysis(serie):
    s=serie.isna().value_counts()    
    if s.size==2:
          serie=serie.dropna()
    elif (s.size==1)&(s[0]==True):
      print('no existen datos')
    datetime_object = datetime.datetime.now()
    lista=[]    
    for u_text in serie:
        if(type(u_text)!= type(datetime_object)):            
            lista.append(u_text.encode("utf-8"))
    text_string_type=', '.join(lista)
    text_string_type=unicode(text_string_type, "utf-8").lower()    
    
    useless_words=nltk.corpus.stopwords.words("spanish")+nltk.corpus.stopwords.words("english")+list(string.punctuation)+[u'para',u'\xa1que',u'\xa1to',u'son',u'con',u'la',u'se',u'en',u'el',u'que',u'la',u'las',u'lo',u'los',u'de',u'the',u'us',u'we',u'A',u'nbsp',u'8217',u'8211',u'8220',u'8221',u'\u2019',u'\u201d',u'\u201c',u'pa\xeds',u'visi\xf3n',u'ray\xf3n',u'pa\xeds',u'tecnolog\xeda',u'ay\xfadanos',u'\xa1que',u'harder\u2014but',u'michoac\xe1n',u'let\xb4s',u'\xa1to',u'granny\xb4s',u'espa\xf1ol.',u'.']#+list(u"\u2019")+list('...')    
    words=nltk.word_tokenize(text_string_type)
    words_filtered=[word for word in words if not word in useless_words]
    
    words_counter=Counter(words_filtered)
    most_common_words = words_counter.most_common()
    return most_common_words

## Obtención de short desciption en Español

In [204]:
# Proyectos con recaudación cero y recaudación igual o superior al objetivo
short_text_cero_donations=data_cero_donation['Short DescSP']
short_text_goal_donations=data_goal_donation['Short DescSP']

In [205]:
short_text_cero_donations=short_text_cero_donations.dropna()
short_text_goal_donations=short_text_goal_donations.dropna()

In [189]:
short_text_cero_donations.shape

(132,)

In [206]:
short_text_goal_donations.shape

(131,)

In [190]:
#short_text_cero_donations y short_text_goal_donations

In [210]:
mcw_cero_donations=text_analysis(short_text_cero_donations)
mcw_goal_donations=text_analysis(short_text_goal_donations)

In [211]:
def transform_words(serie):
    result=[]
    for s in serie:
        #s[0]=s[0].encode('utf-8')
        print(s[0])
        result.append((s[0].encode('utf-8'),s[1]))
        #string.replace(s[0].decode('latin1'), u'\xf1', 'ñ')
        #string.replace(s(0), u'\xf3', 'ó')
        #string.replace(s(0), u'\xed', 'í')
        #string.replace(s(0), u'\xbf', '¿')
        #string.replace(s(0), u'\xe9', 'é')
        #string.replace(s(0), u'\xe1', 'á')
        #string.replace(s(0), u'\xfa', 'ú')
    return result

In [212]:
mcw_cero_donations

[(u'ni\xf1os', 15),
 (u'comunidad', 14),
 (u'personas', 13),
 (u'mujeres', 13),
 (u'vida', 12),
 (u'familias', 10),
 (u'proyecto', 10),
 (u'educaci\xf3n', 9),
 (u'apoyo', 9),
 (u'recursos', 9),
 (u'ayuda', 7),
 (u'salud', 7),
 (u'ni\xf1as', 7),
 (u'estudiantes', 7),
 (u'rurales', 6),
 (u'derechos', 6),
 (u'j\xf3venes', 6),
 (u'crear', 6),
 (u'hacer', 6),
 (u'discapacidad', 6),
 (u'8230', 5),
 (u'mejorar', 5),
 (u'cl\xednica', 5),
 (u'comunidades', 5),
 (u'latinos', 5),
 (u'vamos', 5),
 (u'parte', 5),
 (u'amor', 5),
 (u'oportunidades', 5),
 (u'tener', 4),
 (u'ayudar', 4),
 (u'social', 4),
 (u'diferencia', 4),
 (u'tratamientos', 4),
 (u'necesitamos', 4),
 (u'vida.', 4),
 (u'escuela', 4),
 (u'vidas', 4),
 (u'gran', 4),
 (u'ser', 4),
 (u'puedan', 4),
 (u'queremos', 4),
 (u'violencia', 4),
 (u'poder', 4),
 (u'inmigrantes', 4),
 (u'familia', 4),
 (u'educativo', 3),
 (u'necesita', 3),
 (u'dental', 3),
 (u'enfermedad', 3),
 (u'\xbfqu\xe9', 3),
 (u'vivir', 3),
 (u'escasos', 3),
 (u'construir', 

In [213]:
mcw_goal_donations

[(u'ni\xf1os', 21),
 (u'j\xf3venes', 13),
 (u'educaci\xf3n', 11),
 (u'familias', 11),
 (u'comunidad', 9),
 (u'latinos', 9),
 (u'm\xe9xico', 8),
 (u'comunidades', 8),
 (u'trav\xe9s', 8),
 (u'ayuda', 8),
 (u'acceso', 7),
 (u'mujeres', 7),
 (u'vida', 7),
 (u'apoyar', 7),
 (u'inmigrantes', 7),
 (u'ciudad', 6),
 (u'apoyo', 6),
 (u'recursos', 6),
 (u'latino', 6),
 (u'servicios', 6),
 (u'estudiantes', 6),
 (u'vidas', 5),
 (u'futuro', 5),
 (u'guatemala', 5),
 (u'programa', 5),
 (u'children', 4),
 (u'pobreza', 4),
 (u'provide', 4),
 (u'v\xedctimas', 4),
 (u'teatro', 4),
 (u'as\xed', 4),
 (u'escuela', 4),
 (u'habilidades', 4),
 (u'ofrece', 4),
 (u'fondos', 4),
 (u'empoderar', 4),
 (u'hacer', 4),
 (u'latina', 4),
 (u'nuevo', 4),
 (u'transformar', 4),
 (u'students', 4),
 (u'\xa1ay\xfadanos', 3),
 (u'vez', 3),
 (u'young', 3),
 (u'intensive', 3),
 (u'program', 3),
 (u'agua', 3),
 (u'comunitario', 3),
 (u'givingtuesday', 3),
 (u'vamos', 3),
 (u'construyendo', 3),
 (u'mejorar', 3),
 (u'violencia', 3),

In [221]:
## Creación de base de datos para realizar gráficos de la short description en Español
df_mcw_goal_donations=pd.DataFrame(mcw_goal_donations)
df_mcw_goal_donations.to_excel('data/short_description_words_goal_ESP.xlsx')

In [222]:
df_mcw_cero_donations=pd.DataFrame(mcw_cero_donations)
df_mcw_cero_donations.to_excel('data/short_description_words_cero_ESP.xlsx')

In [223]:
# Se debe continuar con la long description